In [6]:
## Imports
# Data processing
import pandas as pd
import numpy as np
import scipy.stats
from sklearn.metrics.pairwise import cosine_similarity

In [105]:
# Read datasets

users_colnames=['userId', 'gender', 'age', 'profession']
movies_colnames=['movieId', 'year', 'title']
ratings_colnames=['userId', 'movieId', 'rating']

users = pd.read_csv('cse2525-reccommender-systems-challenge/users.csv', sep=';', names=users_colnames)
movies = pd.read_csv('cse2525-reccommender-systems-challenge/movies_v2.csv', sep=';', names=movies_colnames) # changed line 3601 from dataset : ';' -> '_'
ratings = pd.read_csv('cse2525-reccommender-systems-challenge/ratings.csv', sep=';', names=ratings_colnames)

# Check
users.head()
movies.head()
ratings.head()


,userId,gender,age,profession
0,1,F,1,10
1,2,M,56,16
2,3,M,25,15
3,4,M,45,7
4,5,M,25,20


In [50]:
# Count the # of ratings per movie
ratings_by_movie_id = ratings.groupby(['movieId'])['movieId']\
    .count()\
    .reset_index(name='counts')

ratings_by_movie_id.head()


,movieId,counts
0,1,1896
1,2,635
2,3,443
3,4,155
4,5,270


In [97]:
# Number of rows remaining after saving only the ones that have `>= threshold` ratings
movies_threshold = 100
ratings_by_movie_id_processed = ratings_by_movie_id[ratings_by_movie_id['counts'] >= movies_threshold] # drop in another variable
len(ratings_by_movie_id_processed)


1940

In [98]:
# Titles of the most popular (in # of ratings) movies
relevant_movie_ids = ratings_by_movie_id.sort_values(by='counts', ascending=False)['movieId'].array
# relevant_movie_ids.array
# type(relevant_movie_ids.array)
joined_ids = list(set(movies['movieId']) & set(relevant_movie_ids))
joined_ids = [x - 1 for x in joined_ids] # 0-indexed
movies.loc[joined_ids]

,movieId,year,title
0,1,1995,Toy_Story_(1995)
1,2,1995,Jumanji_(1995)
2,3,1995,Grumpier_Old_Men_(1995)
3,4,1995,Waiting_to_Exhale_(1995)
4,5,1995,Father_of_the_Bride_Part_II_(1995)
...,...,...,...
3689,3690,1943,"Phantom_of_the_Opera,_The_(1943)"
3690,3691,1984,Runaway_(1984)
3701,3702,2000,Meet_the_Parents_(2000)
3702,3703,2000,Requiem_for_a_Dream_(2000)


In [99]:
# Create utility (user-movie) matrix
# rows - users, cols - movies
matrix = ratings.pivot_table(index='userId', columns='movieId', values='rating')
print(type(matrix))
matrix.head()

<class 'pandas.core.frame.DataFrame'>


movieId,1,2,3,4,5,6,7,8,9,10,...,3697,3698,3699,3700,3701,3702,3703,3704,3705,3706
userId,,,,,,,,,,,,,,,,,,,,,
1,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [96]:
# Statistics on ratings values

print('The ratings dataset has', ratings['userId'].nunique(), 'unique users')
print('The ratings dataset has', ratings['movieId'].nunique(), 'unique movies')
print('The ratings dataset has', ratings['rating'].nunique(), 'unique ratings')
print('The unique ratings are', sorted(ratings['rating'].unique()))

The ratings dataset has 6040 unique users
The ratings dataset has 3695 unique movies
The ratings dataset has 5 unique ratings
The unique ratings are [1, 2, 3, 4, 5]


In [107]:
# Statistics on users values

print('The users dataset has', users['userId'].nunique(), 'unique users')
print('The users dataset has',users['gender'].nunique(), 'unique genders')
print('The users dataset has', users['age'].nunique(), 'unique age values')
print('The users dataset has', users['profession'].nunique(), 'unique professions values')

The users dataset has 6040 unique users
The users dataset has 2 unique genders
The users dataset has 7 unique age values
The users dataset has 21 unique professions values


In [108]:
# Statistics on movies values

print('The movies dataset has', movies['movieId'].nunique(), 'unique ids')
print('The movies dataset has',movies['year'].nunique(), 'unique year values')
print('The movies dataset has', movies['title'].nunique(), 'unique titles')
print('The movies dataset has year values in range', movies['year'].min(), ' - ', movies['year'].max())

The movies dataset has 3706 unique ids
The movies dataset has 82 unique year values
The movies dataset has 3688 unique titles
The movies dataset has year values in range 0  -  2000


In [101]:
# Data normalization step
# Rating > user_avg => positive value; otherwise negative value
matrix_norm = matrix.subtract(matrix.mean(axis=1), axis = 'rows')
matrix_norm.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,3697,3698,3699,3700,3701,3702,3703,3704,3705,3706
userId,,,,,,,,,,,,,,,,,,,,,
1,0.808511,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [102]:
# User similarity matrix using alternative #1: Pearson correlation
user_similarity = matrix_norm.T.corr()
user_similarity.head()

userId,1,2,3,4,5,6,7,8,9,10,...,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
userId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.642857,-0.562500,0.333333,-0.172516,1.139606e-01,NaN,-0.583333,0.644094,0.163446,...,-0.198030,0.852803,1.000000,NaN,0.381246,-0.150021,-0.394771,NaN,0.061199,-5.062003e-17
2,0.642857,1.000000,-0.208013,0.220863,-0.102728,2.762638e-17,0.262600,-0.015445,0.135250,-0.105465,...,-0.750000,0.338062,0.326183,NaN,0.338558,0.381157,0.183019,-0.5,0.483046,3.193962e-02
3,-0.562500,-0.208013,1.000000,0.774597,-0.437621,-3.492151e-01,0.790569,-0.528594,0.108465,0.016264,...,-0.500000,-0.342997,-0.711556,NaN,0.559017,0.158237,-0.041345,1.0,0.315063,-4.678087e-01
4,0.333333,0.220863,0.774597,1.000000,0.000000,NaN,-0.207514,0.534522,0.866025,-0.371479,...,-0.645497,NaN,-0.038576,NaN,-0.013558,0.489046,0.301511,-1.0,0.000000,5.358259e-01
5,-0.172516,-0.102728,-0.437621,0.000000,1.000000,-7.211103e-01,-0.216966,0.168732,-0.033128,0.021635,...,0.073324,1.000000,0.116775,NaN,0.514356,0.142739,-0.074062,-1.0,1.000000,3.254070e-01


In [103]:
# User similarity matrix using alternative #2: cosine_similarity
user_similarity_cosine = cosine_similarity(matrix_norm.fillna(0))
user_similarity_cosine

array([[ 1.        ,  0.03565432, -0.04347203, ...,  0.        ,
         0.0138397 ,  0.00903407],
       [ 0.03565432,  1.        , -0.01496278, ..., -0.01598902,
         0.02835177, -0.00176974],
       [-0.04347203, -0.01496278,  1.        , ...,  0.0414533 ,
         0.05292534, -0.04647062],
       ...,
       [ 0.        , -0.01598902,  0.0414533 , ...,  1.        ,
         0.04679397, -0.03514554],
       [ 0.0138397 ,  0.02835177,  0.05292534, ...,  0.04679397,
         1.        ,  0.05144363],
       [ 0.00903407, -0.00176974, -0.04647062, ..., -0.03514554,
         0.05144363,  1.        ]])